In [ ]:
!pip install tensorflow

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
print(tf.__version__)

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
database = input_data.read_data_sets('/content/data', one_hot = True)

In [ ]:
lr = 0.001
epochs = 30000
batch_size = 32
image_dimension = 784
neural_network_dimension = 512 #for the hidden layer
latent_variable_dimension = 2

In [1]:
def xavier(in_shape):
  val = tf.random_normal(shape = in_shape, stddev = 1./tf.sqrt(in_shape[0]/2.))
  return val

In [ ]:
 #weight dict

 Weight = {
     "weight_matrix_encoder_hidden": tf.Variable(xavier([image_dimension, neural_network_dimension])),
     "weight_mean_hidden": tf.Variable(xavier([neural_network_dimension, latent_variable_dimension])),
     "weight_std_hidden": tf.Variable(xavier([neural_network_dimension, latent_variable_dimension])),
     "weight_matrix_decoder_hidden": tf.Variable(xavier([latent_variable_dimension, neural_network_dimension])),
     "weight_decoder": tf.Variable(xavier([neural_network_dimension, image_dimension]))
 }

In [ ]:
 Bias = {
     "bias_matrix_encoder_hidden": tf.Variable(xavier([neural_network_dimension])),
     "bias_mean_hidden": tf.Variable(xavier([latent_variable_dimension])),
     "bias_std_hidden": tf.Variable(xavier([latent_variable_dimension])),
     "bias_matrix_decoder_hidden": tf.Variable(xavier([neural_network_dimension])),
     "bias_decoder": tf.Variable(xavier([image_dimension]))
 }

In [ ]:
#computational graph

#Encoder

image_X = tf.placeholder(tf.float32, shape = [None, image_dimension])

Encoder_layer = tf.add(tf.matmul(image_X, Weight['weight_matrix_encoder_hidden']), Bias['bias_matrix_encoder_hidden'])
Encoder_layer = tf.nn.tanh(Encoder_layer)

Mean_layer = tf.add(tf.matmul(Encoder_layer, Weight['weight_mean_hidden']), Bias['bias_mean_hidden'])
Standard_deviation_layer = tf.add(tf.matmul(Encoder_layer, Weight['weight_std_hidden']), Bias['bias_std_hidden']) 


epsilon = tf.random_normal(tf.shape(Standard_deviation_layer), dtype = tf.float32, mean = 0.0, stddev = 1.0)
latent_layer = Mean_layer + tf.exp(0.5 * Standard_deviation_layer) * epsilon  #this is a numerical stability move

Decoder_hidden = tf.add(tf.matmul(latent_layer, Weight['weight_matrix_decoder_hidden']), Bias['bias_matrix_decoder_hidden'])
Decoder_hidden = tf.nn.tanh(Decoder_hidden)

Decoder_output_layer = tf.add(tf.matmul(Decoder_hidden, Weight['weight_decoder']), Bias['bias_decoder'])
Decoder_output_layer = tf.nn.sigmoid(Decoder_output_layer)

In [ ]:
 def loss_function(original_image, reconstructed_image):

   data_fidelity_loss = original_image * tf.log(1e-10 + reconstructed_image) + (1 - original_image) * tf.log(1e-10 + 1 - reconstructed_image)
   data_fidelity_loss = -tf.reduce_sum(data_fidelity_loss, 1)

   KL_div_loss = 1 + Standard_deviation_layer - tf.square(Mean_layer) - tf.exp(Standard_deviation_layer)
   KL_div_loss = -0.5 * tf.reduce_sum(KL_div_loss, 1)

   #weight for each case possible
   alpha = 1
   beta = 1
   network_loss = tf.reduce_mean(alpha * data_fidelity_loss + beta * KL_div_loss)
   return network_loss

 loss_value = loss_function(image_X, Decoder_output_layer)
 optimizer = tf.train.RMSPropOptimizer(lr).minimize(loss_value)

 init = tf.global_variables_initializer()  

In [ ]:
sess = tf.Session()
sess.run(init)

for i in range(epochs):
  x_batch, _ = database.train.next_batch(batch_size)
  _, loss = sess.run([optimizer, loss_value], feed_dict = {image_X : x_batch})
  if i%5000 == 0:
    print('Loss is {0} at iteration {1}'.format(loss, i)

In [ ]:
#testing phase - test only generator to sample the data from the distn but a new sample

 noise_X = tf.placeholder(tf.float32, shape = [None, latent_variable_dimension])

 #generative model testing

Decoder_hidden = tf.add(tf.matmul(noise_X, Weight['weight_matrix_decoder_hidden']), Bias['bias_matrix_decoder_hidden'])
Decoder_hidden = tf.nn.tanh(Decoder_hidden)

Decoder_output_layer = tf.add(tf.matmul(Decoder_hidden, Weight['weight_decoder']), Bias['bias_decoder'])
Decoder_output_layer = tf.nn.sigmoid(Decoder_output_layer)

In [ ]:
#Visualising the output obtained ----

n = 20 #the grid dim
x_limit = np.linspace(-2, 2, n)
y_limit = np.linspace(-2, 2, n)
empty_image = np.empty((28*n, 28*n))

for i, zi in enumerate(x_limit):
  for j, pi in enumerate(y_limit):
    generated_latent_layer = np.array([[zi, pi]]*batch_size) #images fed as input
    #generated_latent_layer = np.random.normal(0, 1, size = [batch_size, latent_variable_dimension])
    generated_image  = sess.run(Decoder_output_layer, feed_dict = {noise_X : generated_latent_layer})
    empty_image[(n-1-i)*28 : (n-i)*28, j*28 : (j+1)*28] = generated_image[0].reshape(28, 28)

plt.figure(figsize = (10, 10))

X, y = np.meshgrid(x_limit, y_limit)
plt.imshow(empty_image, origin = 'upper', cmap = 'gray')
plt.grid(False)
plt.show()